In [2]:
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


### I create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point will be the humidity in each city.

In [5]:
%%capture --no-display
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

# I narrow down cities that fit criteria and drop any results with null values

In [7]:
cities_narrow_df = city_data_df[(city_data_df['Max Temp']>20)]

cities_ideal = cities_narrow_df.dropna()

cities_ideal.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236
10,10,colorado,-22.8375,-51.9731,20.36,86,96,4.69,BR,1666107965
12,12,trairi,-3.2778,-39.2689,31.07,51,34,8.09,BR,1666108038


### I Create a new DataFrame called `hotel_df`to store the city, country, coordinates, and humidity. I also added an empty column, "Hotel Name," to the DataFrame so I can store the hotel found using the Geoapify API

In [9]:
hotel_df=cities_ideal[['City','Country','Lat','Lng','Humidity']].copy()

hotel_df['Hotel Name']=''

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
7,carnarvon,AU,-24.8667,113.6333,76,
9,saint-pierre,RE,-21.3393,55.4781,68,
10,colorado,BR,-22.8375,-51.9731,86,
12,trairi,BR,-3.2778,-39.2689,51,


### For each city, I use the Geoapify API to find the first hotel located within 10,000 metres of coordinates.

In [11]:
api_key = "ad9ff1c83e284a749a23e9b0dd6b6639"
latitude = 38.5167
longitude = -28.7000	
radius = 20000
params = {
    "api_key": api_key,
    "limit": 1,
    "filter": f"circle:{longitude},{latitude},{radius}",
    "bias": f"proximity:{longitude},{latitude}",
    "categories": "accommodation.hotel"
}

# Base URL for the Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

# Make the API request
response = requests.get(base_url, params=params)

# Convert the response to JSON format
name_address = response.json()

# Print the output
print(json.dumps(name_address, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Hospedaria JSF",
                "country": "Portugal",
                "country_code": "pt",
                "city": "Horta",
                "postcode": "9900-038",
                "district": "Horta (Ang\u00fastias)",
                "suburb": "Matriz",
                "street": "Rua Antiga Canada do Porto Pim",
                "iso3166_2": "PT-20",
                "lon": -28.632987399999998,
                "lat": 38.5301319,
                "formatted": "Hospedaria JSF, Rua Antiga Canada do Porto Pim, 9900-038 Horta, Portugal",
                "address_line1": "Hospedaria JSF",
                "address_line2": "Rua Antiga Canada do Porto Pim, 9900-038 Horta, Portugal",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "build

In [12]:
radius = 10000
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    params = {
        "api_key": api_key,
        "limit": 20,
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}",
        "categories": "accommodation.hotel"
    }

    # base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary, Convert the API response to JSON format
    response = requests.get(base_url, params=params)
    name_address = response.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        if name_address["features"]:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

hotel_df.head()

Starting hotel search
faya - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
trairi - nearest hotel: Pousada Villa Aurora
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: Chez Bianca & Benoit
bambous virieux - nearest hotel: Otentic Eco Tent Experience
saint-louis - nearest hotel: No hotel found
cape town - nearest hotel: Townhouse Hotel
naberera - nearest hotel: No hotel found
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
sao joao da barra - nearest hotel: Pousada Mediterrâneo
kapaa - nearest hotel: Pono Kai Resort
mogadishu - nearest hotel: Villa Rays Restaurant & Hotel
hilo - nearest hotel: Dolphin Bay Hotel
businga - nearest hotel: Hôtel Roho
lasa - nearest hotel: Hotel Sonne
vaini - nearest hotel: Ke

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
7,carnarvon,AU,-24.8667,113.6333,76,No hotel found
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
10,colorado,BR,-22.8375,-51.9731,86,Servilha Park Hotel
12,trairi,BR,-3.2778,-39.2689,51,Pousada Villa Aurora


### I added the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

hotel_map= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)
    
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)